In [1]:
import requests
import json
import pandas as pd

In [ ]:
# Testing with the original parameters

In [21]:
# local url
url = 'http://127.0.0.1:5000/'

# sample data
feature_data = {
    'accommodates': 2,
    'bedrooms': 1.0,
    'cleaning_fee': 60.0,
    'extra_people': 15.0,
    'guests_included': 2,
    'minimum_nights': 10,
    'neighbourhood_group_cleansed_Mitte': 1,
    'neighbourhood_group_cleansed_Pankow': 0,
    'neighbourhood_group_cleansed_Tempelhof - Schöneberg': 0,
    'neighbourhood_group_cleansed_Friedrichshain-Kreuzberg': 0,
    'neighbourhood_group_cleansed_Neukölln': 0,
    'neighbourhood_group_cleansed_Charlottenburg-Wilm.': 0,
    'neighbourhood_group_cleansed_Treptow - Köpenick': 0,
    'neighbourhood_group_cleansed_Steglitz - Zehlendorf': 0,
    'neighbourhood_group_cleansed_Reinickendorf': 0,
    'neighbourhood_group_cleansed_Lichtenberg': 0,
    'neighbourhood_group_cleansed_Marzahn - Hellersdorf': 0,
    'neighbourhood_group_cleansed_Spandau': 0, 
    'property_type_Guesthouse': 1,
    'property_type_Apartment': 0,
    'property_type_Condominium': 0,
    'property_type_Loft': 0,
    'property_type_House': 0,
    'property_type_Serviced apartment': 0,
    'property_type_Townhouse': 0,
    'property_type_Other': 0,
    'property_type_Bed and breakfast': 0,
    'property_type_Guest suite': 0,
    'property_type_Hostel': 0,
    'room_type_Entire home/apt': 1,
    'room_type_Private room': 0,
    'room_type_Shared room': 0,
    'bed_type_Real Bed': 1,
    'bed_type_Sofa\Other': 0,
    'instant_bookable_f': 1,
    'instant_bookable_t': 0,
    'cancellation_policy_strict': 1,
    'cancellation_policy_flexible': 0,
    'cancellation_policy_moderate': 0
}

In [22]:
feature_data = json.dumps(feature_data)

In [23]:
# A response of 200 means everything went alright

post_request = requests.post(url, feature_data)
print(post_request)

<Response [405]>


In [5]:
print(post_request.json())

{'results': {'results': 84}}


In [ ]:
# Testing with hyperparameters

In [33]:
# local url
url = 'http://127.0.0.1:5000/'

# sample data
data = {
    'accommodates': 2,
    'bedrooms': 1.0,
    'cleaning_fee': 60.0,
    'extra_people': 15.0,
    'guests_included': 2,
    'minimum_nights': 10,
    'neighbourhood_group_cleansed_Mitte': 1,
    'neighbourhood_group_cleansed_Pankow': 0,
    'neighbourhood_group_cleansed_Tempelhof - Schöneberg': 0,
    'neighbourhood_group_cleansed_Friedrichshain-Kreuzberg': 0,
    'neighbourhood_group_cleansed_Neukölln': 0,
    'neighbourhood_group_cleansed_Charlottenburg-Wilm.': 0,
    'neighbourhood_group_cleansed_Treptow - Köpenick': 0,
    'neighbourhood_group_cleansed_Steglitz - Zehlendorf': 0,
    'neighbourhood_group_cleansed_Reinickendorf': 0,
    'neighbourhood_group_cleansed_Lichtenberg': 0,
    'neighbourhood_group_cleansed_Marzahn - Hellersdorf': 0,
    'neighbourhood_group_cleansed_Spandau': 0, 
    'property_type_Guesthouse': 1,
    'property_type_Apartment': 0,
    'property_type_Condominium': 0,
    'property_type_Loft': 0,
    'property_type_House': 0,
    'property_type_Serviced apartment': 0,
    'property_type_Townhouse': 0,
    'property_type_Other': 0,
    'property_type_Bed and breakfast': 0,
    'property_type_Guest suite': 0,
    'property_type_Hostel': 0,
    'room_type_Entire home/apt': 1,
    'room_type_Private room': 0,
    'room_type_Shared room': 0,
    'bed_type_Real Bed': 1,
    'bed_type_Sofa\Other': 0,
    'instant_bookable_f': 1,
    'instant_bookable_t': 0,
    'cancellation_policy_strict': 1,
    'cancellation_policy_flexible': 0,
    'cancellation_policy_moderate': 0
}

In [34]:
data = json.dumps(data)

In [35]:
# A response of 200 means everything went alright

post_request = requests.post(url, data)
print(post_request)

<Response [200]>


In [36]:
print(post_request.json())

{'results': {'results': 77}}


In [28]:
import pandas as pd
from flask import Flask, jsonify, request, render_template
import pickle                      ## To use .pkl model
import json                        ## To load in 'features.json'
# import psycopg2                  ## To connect to PostgreSQL Database
# from dotenv import load_dotenv   ## To get .env
# import os                        ## To get .env


# load_dotenv()

# DB_NAME=os.getenv('DB_NAME', default='OOPS')
# DB_USER=os.getenv('DB_USER', default='OOPS')
# DB_PASS=os.getenv('DB_PASS', default='OOPS')
# DB_HOST=os.getenv('DB_HOST', default='OOPS')

# conn = psycopg2.connect(dbname='DB_NAME', user='DB_USER',
#                         password='DB_PASS', host='DB_HOST')
# cur = conn.cursor()


# Load the model from 'berlin_model.pkl' file
model = pickle.load(open('berlin_model.pkl', 'rb'))

# App
app = Flask(__name__)

# Routes
@app.route('/')
def index():
    return render_template('feature_input.html')

@app.route('/price', methods=['POST'])
def predict_price():

    # Takes data entered at index route via the 'feature_input.html' template
    data = dict(request.form)
    
    # The file 'features.json' contains skeleton JSON of feature variables with default
    # values set to '0'.  This block of code compares the data from the request form to
    # the default values from the JSON file and makes changes depending on what the input
    # was.  For features that have multiple permutations (neighbourhood_group_cleansed, etc),
    # whichever was chosen in the drop-down list will be reassigned as an integer(1) while
    # the others remain an integer(0) which denotes that the listing is not a particular
    # feature permutation and thus, can be ran through pickeled model.
    with open('features.json', 'r') as f:
        features_dict = json.load(f)
        for key, value in data.items():
            if key in features_dict:
                features_dict[key] = float(value)
            elif value in features_dict:
                features_dict[value] = 1
        # print(features_dict)
        data = features_dict

    # This code will take the variable 'data' and convert it from JSON into a Pandas
    # Dataframe which can be used with the next line of code
    df = pd.json_normalize(data)

    # Prediction based on the Dataframe containing feature values
    results = int(model.predict(df)[0])

    # Return the predicted price as JSON
    # return jsonify(results=results)
      return jsonify(price=results)
    # return df

if __name__ == "__main__":
    app.run(port=5000, debug=True)


NameError: name 'predict_price' is not defined